In [5]:
class Graph:
    def __init__(self, vertices=None):
        self.graph = {}
        self.vertices = []
        if vertices:
            for vertex in vertices:
                self.add_vertex(vertex)
    
    def add_vertex(self, vertex):
        if vertex not in self.graph:
            self.graph[vertex] = []
            self.vertices.append(vertex)
    
    def add_edge(self, u, v):
        if u in self.graph and v in self.graph:
            self.graph[u].append(v)
        else:
            if u not in self.graph:
                self.add_vertex(u)
            if v not in self.graph:
                self.add_vertex(v)
            self.graph[u].append(v)
    
    def get_edges(self):
        edges = []
        for u in self.graph:
            for v in self.graph[u]:
                edges.append((u, v))
        return edges


# 1. Topological Sort
def topological_sort(graph):

    result = []
    visited = set()
    temp = set()  # For cycle detection
    
    def visit(vertex):
        if vertex in temp:
            raise ValueError("Graph has a cycle, topological sort not possible")
        if vertex not in visited:
            temp.add(vertex)
            for neighbor in graph.graph.get(vertex, []):
                visit(neighbor)
            temp.remove(vertex)
            visited.add(vertex)
            result.append(vertex)
    
    for vertex in graph.vertices:
        if vertex not in visited:
            visit(vertex)
    
    return result[::-1]  # Reverse the result to get correct order


# 2. Depth-First Search
def dfs(graph, start, finish_times=None):

    visited = set()
    discovery = []
    time = [0] 
    
    if finish_times is None:
        finish_times = {}
    
    def dfs_visit(vertex):
        visited.add(vertex)
        discovery.append(vertex)
        time[0] += 1 
        
        for neighbor in graph.graph.get(vertex, []):
            if neighbor not in visited:
                dfs_visit(neighbor)
        
        time[0] += 1
        finish_times[vertex] = time[0]
    
    dfs_visit(start)
    
    return discovery, finish_times


def dfs_full(graph):

    visited = set()
    discovery = []
    finish_times = {}
    time = [0]
    
    def dfs_visit(vertex):
        visited.add(vertex)
        discovery.append(vertex)
        time[0] += 1  
        
        for neighbor in graph.graph.get(vertex, []):
            if neighbor not in visited:
                dfs_visit(neighbor)
        
        time[0] += 1
        finish_times[vertex] = time[0]
    
    for vertex in graph.vertices:
        if vertex not in visited:
            dfs_visit(vertex)
    
    return discovery, finish_times


# 3. Kruskal's Algorithm
def kruskal(graph, weights):

    edges = []
    for u in graph.graph:
        for v in graph.graph[u]:
            if (v, u) not in [(e[1], e[0]) for e in edges]:
                edges.append((u, v))
    
    edges.sort(key=lambda e: weights.get((e[0], e[1]), float('inf')))
    
    parent = {vertex: vertex for vertex in graph.vertices}
    
    def find(vertex):
        if parent[vertex] != vertex:
            parent[vertex] = find(parent[vertex]) 
        return parent[vertex]
    
    def union(u, v):
        parent[find(u)] = find(v)
    
    mst = []
    for u, v in edges:
        if find(u) != find(v): 
            union(u, v)
            mst.append((u, v))
    
    return mst


# Test the algorithms
def test_clothing_example():
    clothing_graph = Graph()
    
    clothing_items = {
        "undershorts": "11/16",
        "pants": "12/15",
        "belt": "6/7",
        "shirt": "1/8",
        "tie": "2/5",
        "jacket": "3/4",
        "socks": "17/18",
        "shoes": "13/14",
        "watch": "9/10"
    }
    
    for item in clothing_items:
        clothing_graph.add_vertex(item)
    
    clothing_graph.add_edge("undershorts", "pants")
    clothing_graph.add_edge("undershorts", "shoes")
    clothing_graph.add_edge("pants", "shoes")
    clothing_graph.add_edge("pants", "belt")
    clothing_graph.add_edge("belt", "jacket")
    clothing_graph.add_edge("shirt", "belt")
    clothing_graph.add_edge("shirt", "tie")
    clothing_graph.add_edge("tie", "jacket")
    clothing_graph.add_edge("socks", "shoes")
    
    print("Clothing Graph:")
    for v in clothing_graph.graph:
        print(f"{v} -> {clothing_graph.graph[v]}")
    
    # 1. Test Topological Sort
    print("\n1. Topological Sort:")
    topo_order = topological_sort(clothing_graph)
    print(topo_order)
    
    print("\nExpected order (from left to right in part b):")
    expected_order = ["socks", "undershorts", "pants", "shoes", "watch", "shirt", "belt", "tie", "jacket"]
    print(expected_order)
    
    # 2. Test DFS
    print("\n2. Depth-First Search:")
    
    discovery_times = {}
    finish_times = {}
    for item, time_label in clothing_items.items():
        discovery, finish = map(int, time_label.split('/'))
        discovery_times[item] = discovery
        finish_times[item] = finish
    
    sorted_by_discovery = sorted(discovery_times.items(), key=lambda x: x[1])
    start_vertex = sorted_by_discovery[0][0]
    
    print(f"Starting DFS from vertex with earliest discovery time: {start_vertex}")
    discovery, calculated_finish = dfs(clothing_graph, start_vertex)
    print(f"DFS Discovery Order: {discovery}")
    print("Calculated Finish Times:", calculated_finish)
    print("Expected Finish Times:", finish_times)
    
    print("\nRunning Full DFS on all vertices:")
    full_discovery, full_finish = dfs_full(clothing_graph)
    print(f"Full DFS Discovery Order: {full_discovery}")
    
    # 3. Test Kruskal's Algorithm
    print("\n3. Kruskal's Algorithm:")
    
    undirected_graph = Graph(clothing_graph.vertices)
    for u in clothing_graph.graph:
        for v in clothing_graph.graph[u]:
            undirected_graph.add_edge(u, v)
            undirected_graph.add_edge(v, u) 
    
    weights = {}
    for u in undirected_graph.graph:
        for v in undirected_graph.graph[u]:
            # Assign weight based on sum of the discovery times (just for demonstration)
            weights[(u, v)] = discovery_times[u] + discovery_times[v]
    
    mst = kruskal(undirected_graph, weights)
    print("Minimum Spanning Tree Edges:")
    for edge in mst:
        print(f"{edge[0]} -- {edge[1]} (Weight: {weights[edge]})")
    
    # Calculate total weight of MST
    total_weight = sum(weights[edge] for edge in mst)
    print(f"Total MST Weight: {total_weight}")


if __name__ == "__main__":
    test_clothing_example()

Clothing Graph:
undershorts -> ['pants', 'shoes']
pants -> ['shoes', 'belt']
belt -> ['jacket']
shirt -> ['belt', 'tie']
tie -> ['jacket']
jacket -> []
socks -> ['shoes']
shoes -> []
watch -> []

1. Topological Sort:
['watch', 'socks', 'shirt', 'tie', 'undershorts', 'pants', 'belt', 'jacket', 'shoes']

Expected order (from left to right in part b):
['socks', 'undershorts', 'pants', 'shoes', 'watch', 'shirt', 'belt', 'tie', 'jacket']

2. Depth-First Search:
Starting DFS from vertex with earliest discovery time: shirt
DFS Discovery Order: ['shirt', 'belt', 'jacket', 'tie']
Calculated Finish Times: {'jacket': 4, 'belt': 5, 'tie': 7, 'shirt': 8}
Expected Finish Times: {'undershorts': 16, 'pants': 15, 'belt': 7, 'shirt': 8, 'tie': 5, 'jacket': 4, 'socks': 18, 'shoes': 14, 'watch': 10}

Running Full DFS on all vertices:
Full DFS Discovery Order: ['undershorts', 'pants', 'shoes', 'belt', 'jacket', 'shirt', 'tie', 'socks', 'watch']

3. Kruskal's Algorithm:
Minimum Spanning Tree Edges:
shirt --